<a href="https://colab.research.google.com/github/dhatchayeni-ai/customer-support-ticket-triage/blob/main/Week1_Customer_Support_Triage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv("/content/customer_support_tickets.csv")

print("Loaded:", df.shape)
print("Columns:", df.columns.tolist())
print("Missing values:\n", df.isnull().sum())
print("Sample data:\n", df.head(3))


Loaded: (8469, 17)
Columns: ['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age', 'Customer Gender', 'Product Purchased', 'Date of Purchase', 'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status', 'Resolution', 'Ticket Priority', 'Ticket Channel', 'First Response Time', 'Time to Resolution', 'Customer Satisfaction Rating']
Missing values:
 Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
d

In [3]:
import pandas as pd
import re
from datetime import datetime, timedelta


df = pd.read_csv("/content/customer_support_tickets.csv")


text_col = df.select_dtypes(include="object").columns[0]

print("Rows, Columns:", df.shape)
print("Using text column:", text_col)
print("Missing values:\n", df.isnull().sum())
print("\nSample tickets:\n", df[text_col].head(3))

def clean_text(t):
    t = str(t).lower()
    t = re.sub(r"[^a-z0-9\s]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

df["clean_message"] = df[text_col].apply(clean_text)

def classify_issue(t):
    if any(k in t for k in ["payment", "card", "upi", "transaction"]):
        return "PAYMENT"
    elif any(k in t for k in ["login", "signin", "password", "otp"]):
        return "LOGIN"
    elif any(k in t for k in ["delivery", "shipment", "courier", "late"]):
        return "DELIVERY"
    elif any(k in t for k in ["refund", "return", "money back"]):
        return "REFUND"
    elif any(k in t for k in ["bug", "error", "crash", "not working"]):
        return "BUG"
    else:
        return "GENERAL"

df["issue_type"] = df["clean_message"].apply(classify_issue)

def assign_priority(t):
    if any(k in t for k in ["fraud", "charged twice", "payment failed"]):
        return "P0"
    elif any(k in t for k in ["urgent", "blocked", "unable"]):
        return "P1"
    elif any(k in t for k in ["delay", "slow", "problem"]):
        return "P2"
    else:
        return "P3"

df["priority"] = df["clean_message"].apply(assign_priority)

sla_map = {"P0": 2, "P1": 6, "P2": 24, "P3": 48}
df["sla_hours"] = df["priority"].map(sla_map)

df["created_at"] = datetime.now()
df["due_time"] = df["created_at"] + df["sla_hours"].apply(lambda x: timedelta(hours=x))

print("\n--- SUPPORT MANAGER REPORT ---")
print("Total Tickets:", len(df))
print("Tickets by Issue:\n", df["issue_type"].value_counts())
print("Tickets by Priority:\n", df["priority"].value_counts())
df.to_csv("/content/final_support_tickets.csv", index=False)
print("\nFinal CSV exported: final_support_tickets.csv")


Rows, Columns: (8469, 17)
Using text column: Customer Name
Missing values:
 Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
dtype: int64

Sample tickets:
 0          Marisa Obrien
1           Jessica Rios
2    Christopher Robbins
Name: Customer Name, dtype: object

--- SUPPORT MANAGER REPORT ---
Total Tickets: 8469
Tickets by Issue:
 issue_type
GENERAL    8461
PAYMENT       8
Name: count, dtype: int64
Tickets by Pri